# NOTES:

- We need to find the total amount donated for opportunities.
- Do we proceed with just the donations with 100% certainty? Do we look at estimated amount? Actual amount received?
- We should group by City


## Opportunities dataset:
- We need to do some join operations to get the data together.
    - Join contact by ContactId
    - Group by activity sector to get donations by location
- Calculate opportunities won by location
- Calculate the total amount of money won, and expected to be won by location
- Why are we losing what we are losing. Where are we losing?


## ML Task:
- We need to group by date to get total payments by date. From this, we create the time-series dataset.



In [1]:
import pandas as pd
import os

In [2]:
os.listdir("shared_data_read_only")

['consumer_price_index.xlsx',
 'Items and Opportunities.xlsx',
 'Data dictionary.xlsx',
 'Documentation',
 'Invoice and Opportunities.xlsx',
 'Business Account.xlsx',
 'Opportunities.xlsx',
 'Non-Stock Items.xlsx',
 'Contacts.xlsx']

In [3]:
data = {}
for doc in os.listdir('shared_data_read_only'):
    try:
        data[doc] = pd.read_excel(f"shared_data_read_only/{doc}")
    except:
        continue

In [4]:
data.keys()

dict_keys(['consumer_price_index.xlsx', 'Items and Opportunities.xlsx', 'Data dictionary.xlsx', 'Invoice and Opportunities.xlsx', 'Business Account.xlsx', 'Opportunities.xlsx', 'Non-Stock Items.xlsx', 'Contacts.xlsx'])

In [5]:
data["Contacts.xlsx"]

,ContactID,Active,Business Account,Class,Class Name,Financing preference,Owner,Follow-up with,Postal Code,City,State,Code pays,Country Name,Gender,Client Source,Last Modified On,Created On
0,39405,True,C00091,01BUSIN,Business function,NaN,26.0,NaN,WC1V 6JS,London,NaN,GB,United Kingdom of Great Britain and Northern I...,NaN,NaN,2023-07-11 14:37:02.183,2021-06-03 08:35:32.823
1,41938,True,C06932,03PDG,DG,"Apprenticeship tax, Gala Lyon, But en Or Lyon,...",29.0,NaN,69120,VAULX EN VELIN,NaN,FR,France,NaN,NaN,2023-02-07 12:19:14.043,2022-01-05 16:34:58.807
2,37775,True,C01438,10DIRGENE,General Management,NaN,17.0,8.0,93701,DRANCY,NaN,FR,France,Male,NaN,2021-03-23 07:05:02.557,2021-03-19 06:44:36.557
3,37776,True,C01430,13ELU,Elected,NaN,17.0,8.0,93000,BOBIGNY,NaN,FR,France,Male,NaN,2021-03-23 07:05:03.410,2021-03-19 06:44:37.070
4,36942,True,C01204,13ELU,Elected,NaN,6.0,5.0,69140,RILLIEUX-LA-PAPE,NaN,FR,France,Male,NaN,2021-04-13 10:52:53.050,2021-03-17 16:45:45.083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5506,38255,True,C01453,11DIRTER,Territorial Management,NaN,40.0,NaN,59100,ROUBAIX,NaN,FR,France,Female,NaN,2021-03-19 06:51:37.223,2021-03-19 06:51:37.223
5507,39179,True,C01884,01BUSIN,Business function,"Apprenticeship tax, Annual report",7.0,NaN,75010,PARIS,NaN,FR,France,Female,NaN,2021-05-17 11:23:30.993,2021-05-17 11:23:30.993
5508,40695,True,C03979,20PART,Individual,NaN,NaN,NaN,75010,Paris,NaN,FR,France,NaN,HelloAsso,2021-10-13 10:53:49.693,2021-10-13 10:53:49.693
5509,43323,True,C07697,02COM,Communication,"JDLV Ceremony, Annual report, Greetings",29.0,NaN,75007,PARIS,NaN,FR,France,NaN,NaN,2022-06-03 14:58:41.643,2022-05-09 13:45:01.623


In [6]:
# Opportunities Prep

data["Opportunities.xlsx"].rename(columns={"Contact ID": "ContactID"}, inplace=True)

# Preprcoessing Opportunities

# Many nan vals in ContactID

In [18]:
data["Opportunities.xlsx"].isna().sum()

Opportunity ID                                  0
Business Account                                2
ContactID                                    3195
Estimated Close Date                            0
Status                                          0
Stage                                           0
Probability                                     0
Estimated amount                                0
Amount received                              3250
Difference                                      0
Owner Name                                    250
Follow-up with                               5001
Class ID                                        0
Class                                           0
Source                                       5445
Source Campaign                              4828
Campaign Name                                4828
Programme financé                            2136
Action financée                              5613
Agreement                                    2100


In [13]:
df[df.ContactID.isna()]

,Opportunity ID,Business Account,ContactID,Estimated Close Date,Status,Stage,Probability,Estimated amount,Amount received,Difference,...,Follow-up withcontacts,Postal Code,City,State,Code pays,Country Name,Gender,Client Source,Last Modified On,Created Oncontacts
38,6349,C21530,NaN,2023-10-10,New,Initial Prospect,20,25000.0,NaN,-25000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
39,6348,C21529,NaN,2023-10-10,New,Initial Prospect,20,150000.0,NaN,-150000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
40,6347,C21528,NaN,2023-10-10,New,Initial Prospect,20,100000.0,NaN,-100000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
42,6345,C01325,NaN,2023-10-10,New,Initial Prospect,20,200000.0,NaN,-200000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
43,6344,C21527,NaN,2023-10-10,New,Initial Prospect,20,50000.0,NaN,-50000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5981,5,C00036,NaN,2020-01-01,Won,Won,100,1059.0,NaN,-1059.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
5982,4,C00024,NaN,2020-01-01,Won,Won,200,7500.0,7500.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
5983,3,C00011,NaN,2020-01-01,Won,Won,100,35000.0,NaN,-35000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
5984,2,C00331,NaN,2020-01-01,Won,Won,100,20000.0,NaN,-20000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT


# Values in Class ID

In [26]:
data["Opportunities.xlsx"]["Class ID"].value_counts()

PPRIV MEC     2523
PPRIV EVNT    1251
PPRIV TA      1118
PPUB           425
CAMP SAL       267
CAMP TER       196
CAMP EVT       168
PPRIV DONN      28
PPRIV MCOM      10
Name: Class ID, dtype: int64

In [32]:
data["Opportunities.xlsx"]["Business Account"]

0       C00037
1       C01254
2       C21512
3       C21509
4       C21508
         ...  
5981    C00036
5982    C00024
5983    C00011
5984    C00331
5985    C00331
Name: Business Account, Length: 5986, dtype: object

In [31]:
data["Business Account.xlsx"]["Business Account"]

0       C00001
1       C00002
2       C00003
3       C00004
4       C00005
         ...  
3478    C19916
3479    C19917
3480    C19918
3481    C19986
3482    C19987
Name: Business Account, Length: 3483, dtype: object

In [36]:
data["Opportunities.xlsx"].dtypes

Opportunity ID                                        int64
Business Account                                     object
ContactID                                           float64
Estimated Close Date                         datetime64[ns]
Status                                               object
Stage                                                object
Probability                                           int64
Estimated amount                                    float64
Amount received                                     float64
Difference                                          float64
Owner Name                                          float64
Follow-up with                                      float64
Class ID                                             object
Class                                                object
Source                                               object
Source Campaign                                     float64
Campaign Name                           

In [38]:
data["Business Account.xlsx"].dtypes

Business Account                   object
Customer Status                    object
Type                               object
Partner type                       object
Class                              object
Class Name                         object
Activity sector                    object
Parent Business Account            object
Owner Name                        float64
Follow-up with                    float64
Postal Code                        object
City                               object
State                              object
Country ID                         object
Country Name                       object
Created On                 datetime64[ns]
Last Modified On           datetime64[ns]
dtype: object

# Join Operation

In [25]:
# Join on Business Accounts

df = data["Opportunities.xlsx"].merge(data["Business Account.xlsx"], on="Business Account", how="left")

In [26]:
df.shape

(5986, 59)

In [28]:
df = df.merge(data["Contacts.xlsx"], on="ContactID", how="left")

In [30]:
all_opportunities = data["Items and Opportunities.xlsx"].merge(data["Invoice and Opportunities.xlsx"], on="Opportunity ID")
all_opportunities = all_opportunities.merge(data["Non-Stock Items.xlsx"], on="Inventory ID")

In [31]:
df = df.merge(all_opportunities, on='Opportunity ID', how="left")

In [33]:
df

,Opportunity ID,Business Account_x,ContactID,Estimated Close Date,Status,Stage,Probability,Estimated amount,Amount received,Difference,...,Amount,Date,Description EN,Posting Class,Tax Category_y,Base Unit,Item Status,Created On_y,Last Modified On,Default Price
0,6387,C00037,35334.0,2023-11-27,New,Won,100,9000.0,NaN,-9000.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
1,6386,C01254,36240.0,2023-10-11,New,Won,100,9000.0,NaN,-9000.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
2,6385,C21512,47640.0,2023-10-11,New,Oral agreement,80,140.0,NaN,-140.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
3,6384,C21509,47636.0,2023-10-11,New,Oral agreement,80,350.0,NaN,-350.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
4,6383,C21508,47635.0,2023-10-10,New,Oral agreement,80,120.0,NaN,-120.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8017,5,C00036,NaN,2020-01-01,Won,Won,100,1059.0,NaN,-1059.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
8018,4,C00024,NaN,2020-01-01,Won,Won,200,7500.0,7500.0,0.0,...,7500.0,2021-07-02,Sponsorship,MECENAT,TVA0,PIECE,Active,2021-02-12 02:59:03.890,2021-09-01 10:39:46.240,0.0
8019,3,C00011,NaN,2020-01-01,Won,Won,100,35000.0,NaN,-35000.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
8020,2,C00331,NaN,2020-01-01,Won,Won,100,20000.0,NaN,-20000.0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN


# function

In [39]:
def load_and_join_data():
    data = {}
    for doc in os.listdir('shared_data_read_only'):
        try:
            data[doc] = pd.read_excel(f"data/shared_data_read_only/{doc}")
        except:
            continue
    data["Opportunities.xlsx"].rename(columns={"Contact ID": "ContactID"}, inplace=True)
    df = data["Opportunities.xlsx"].merge(data["Business Account.xlsx"], on="Business Account", how="left")
    df = df.merge(data["Contacts.xlsx"], on="ContactID", how="left")
    df = df.merge(all_opportunities, on='Opportunity ID', how="left")
    return df

In [41]:
df = load_and_join_data()

In [44]:
df.to_csv("final_data.csv")

Index(['Opportunity ID', 'Business Account_x', 'ContactID',
       'Estimated Close Date', 'Status', 'Stage', 'Probability',
       'Estimated amount', 'Amount received', 'Difference',
       ...
       'Amount', 'Date', 'Description EN', 'Posting Class', 'Tax Category_y',
       'Base Unit', 'Item Status', 'Created On_y', 'Last Modified On',
       'Default Price'],
      dtype='object', length=101)

In [46]:
cd Team-1

/home/jupyter-user-6/Team-1
